In [256]:
import pandas as pd
import pickle
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import gc
%matplotlib inline

In [257]:
url = 'https://www.pro-football-reference.com/players/B/BradTo00/gamelog/'
tb = pd.read_html(url)[0]
tb = tb.iloc[:, :-15]

# making a list of the dual-headers
col_list = [col for col in tb]
col_list2 = []

# only grabbing the sub-header 
for col in col_list:
    col_list2.append(col[1])

# renaming columns as sub-header   
tb.columns = col_list2

# splitting the df into two separate df's
# applying numeric filter to number
# then slamming them back together
clean_tb = pd.concat(
    [tb.iloc[:,:37], tb.iloc[:,37:].apply(pd.to_numeric, errors='coerce')],
    axis=1)

clean_tb = clean_tb[clean_tb["Rk"].str.contains("Rk")==False]
clean_tb.drop(clean_tb.tail(1).index,inplace=True)
clean_tb.columns.values[7] = 'Home'
clean_tb['Home'] = clean_tb['Home'].apply(lambda x: 0 if x == '@' else 1)

clean_tb.columns.values[19] = 'Sk_Yds'
clean_tb.columns.values[22] = 'Ru_Att'
clean_tb.columns.values[23] = 'Ru_Yds'
clean_tb.columns.values[24] = 'Ru_Y/A'
clean_tb.columns.values[25] = 'Ru_TB'
clean_tb.columns.values[26] = 'Re_Tgt'
clean_tb.columns.values[27] = 'Re_Rec'
clean_tb.columns.values[28] = 'Re_Yds'
clean_tb.columns.values[29] = 'Re_Y/R'
clean_tb.columns.values[30] = 'Re_TD'
clean_tb.columns.values[31] = 'Re_Ctch%'
clean_tb.columns.values[32] = 'Re_Y/Tgt'
clean_tb.columns.values[33] = 'Total_NP_TD'
clean_tb = clean_tb.drop(columns= ['Result', 'GS'])


# looking at it in all its glory
clean_tb

,Rk,Year,Date,G#,Week,Age,Tm,Home,Opp,Cmp,...,Re_Rec,Re_Yds,Re_Y/R,Re_TD,Re_Ctch%,Re_Y/Tgt,Total_NP_TD,Pts,Fmb,FL
0,1,2000,2000-09-03,1,1,NaN,NWE,1,TAM,Inactive,...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive
1,2,2000,2000-09-11,2,2,NaN,NWE,0,NYJ,Inactive,...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive
2,3,2000,2000-09-17,3,3,NaN,NWE,1,MIN,Inactive,...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive
3,4,2000,2000-09-24,4,4,NaN,NWE,0,MIA,Inactive,...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive
4,5,2000,2000-10-01,5,5,NaN,NWE,0,DEN,Inactive,...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,351,2021,2021-12-26,15,16,44.145,TAM,0,CAR,18,...,0,0,NaN,0,0.0%,NaN,0,0,0,0
372,352,2021,2022-01-02,16,17,44.152,TAM,0,NYJ,34,...,0,0,NaN,0,0.0%,NaN,0,0,0,0
373,353,2021,2022-01-09,17,18,44.159,TAM,1,CAR,29,...,0,0,NaN,0,0.0%,NaN,0,0,0,0
375,354,2022,2022-09-11,1,1,45.039,TAM,0,DAL,18,...,0,0,NaN,0,0.0%,NaN,0,0,0,0


In [258]:
clean_tb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355 entries, 0 to 376
Data columns (total 35 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Rk           355 non-null    object
 1   Year         355 non-null    object
 2   Date         355 non-null    object
 3   G#           355 non-null    object
 4   Week         355 non-null    object
 5   Age          320 non-null    object
 6   Tm           355 non-null    object
 7   Home         355 non-null    int64 
 8   Opp          355 non-null    object
 9   Cmp          355 non-null    object
 10  Att          355 non-null    object
 11  Cmp%         355 non-null    object
 12  Yds          355 non-null    object
 13  TD           355 non-null    object
 14  Int          355 non-null    object
 15  Rate         355 non-null    object
 16  Sk           355 non-null    object
 17  Sk_Yds       355 non-null    object
 18  Y/A          355 non-null    object
 19  AY/A         355 non-null    

In [259]:
col_list = clean_tb.columns

In [260]:
col_list

Index(['Rk', 'Year', 'Date', 'G#', 'Week', 'Age', 'Tm', 'Home', 'Opp', 'Cmp',
       'Att', 'Cmp%', 'Yds', 'TD', 'Int', 'Rate', 'Sk', 'Sk_Yds', 'Y/A',
       'AY/A', 'Ru_Att', 'Ru_Yds', 'Ru_Y/A', 'Ru_TB', 'Re_Tgt', 'Re_Rec',
       'Re_Yds', 'Re_Y/R', 'Re_TD', 'Re_Ctch%', 'Re_Y/Tgt', 'Total_NP_TD',
       'Pts', 'Fmb', 'FL'],
      dtype='object')

In [261]:
col_list = col_list.drop(['Date', 'Tm', 'Opp'])

In [262]:
col_list

Index(['Rk', 'Year', 'G#', 'Week', 'Age', 'Home', 'Cmp', 'Att', 'Cmp%', 'Yds',
       'TD', 'Int', 'Rate', 'Sk', 'Sk_Yds', 'Y/A', 'AY/A', 'Ru_Att', 'Ru_Yds',
       'Ru_Y/A', 'Ru_TB', 'Re_Tgt', 'Re_Rec', 'Re_Yds', 'Re_Y/R', 'Re_TD',
       'Re_Ctch%', 'Re_Y/Tgt', 'Total_NP_TD', 'Pts', 'Fmb', 'FL'],
      dtype='object')

In [263]:
clean_tb[col_list] = clean_tb[col_list].apply(pd.to_numeric, errors='coerce', axis=1)

In [264]:
clean_tb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355 entries, 0 to 376
Data columns (total 35 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rk           355 non-null    float64
 1   Year         355 non-null    float64
 2   Date         355 non-null    object 
 3   G#           355 non-null    float64
 4   Week         355 non-null    float64
 5   Age          320 non-null    float64
 6   Tm           355 non-null    object 
 7   Home         355 non-null    float64
 8   Opp          355 non-null    object 
 9   Cmp          320 non-null    float64
 10  Att          320 non-null    float64
 11  Cmp%         320 non-null    float64
 12  Yds          320 non-null    float64
 13  TD           320 non-null    float64
 14  Int          320 non-null    float64
 15  Rate         320 non-null    float64
 16  Sk           320 non-null    float64
 17  Sk_Yds       320 non-null    float64
 18  Y/A          320 non-null    float64
 19  AY/A    

In [ ]:
Dome_Dict = {
    "ARI" : 1,
    "ATL" : 1,
    "DAL" : 1,
    "DET" : 1,
    "HOU" : 1,
    "IND" : 1,
    "LAC" : 1,
    "LAR" : 1,
    "LVR" : 1,
    "MIN" : 1,
    "NOR" : 1,
}

In [ ]:
clean_tb['Dome_Home'] = clean_tb['Tm'].apply(lambda x: 1 if x in Dome_Dict.keys() else 0)
clean_tb['Dome_Away'] = clean_tb['Opp'].apply(lambda x: 1 if x in Dome_Dict.keys() else 0)

In [ ]:
clean_tb['Dome'] = clean_tb['Dome_Home'] + clean_tb['Dome_Away']

In [ ]:
clean_tb = clean_tb.drop(columns= ['Dome_Home', 'Dome_Away']

In [242]:
clean_tb['passing_ppr'] = ((clean_tb['Yds'] / 25)) + (clean_tb['TD'] * 4) + (clean_tb['Int'] * -2)
clean_tb['rushing_ppr'] = ((clean_tb['Ru_Yds'] + clean_tb['Re_Yds']) / 10)
clean_tb['td_ppr'] = (clean_tb['Total_NP_TD'] * 6)
clean_tb['receiving_ppr'] = (clean_tb['Re_Rec'] * 1)
clean_tb['fumble_ppr'] = (clean_tb['FL'] * -2)
#clean_tb['total'] = sum(clean_tb[-6:])
clean_tb['Total'] = clean_tb.iloc[:, -6:-1].sum(axis=1)


# 1 point per 25 passing yards
# 4 points per passing touchdown
# -2 points per interception
# 1 point per reception
# 1 point per 10 yards rushing/receiving
# 6 points per TD
# 2 points per two-point conversion
# -2 points per fumble lost


In [243]:
clean_tb.tail(10)

,Rk,Year,Date,G#,Week,Age,Tm,Home,Opp,Cmp,...,Total_NP_TD,Pts,Fmb,FL,passing_ppr,rushing_ppr,td_ppr,receiving_ppr,fumble_ppr,Total
366,346.0,2021.0,2021-11-22,10.0,11.0,44.111,TAM,1.0,NYG,30.0,...,0.0,0.0,0.0,0.0,18.28,1.0,0.0,0.0,-0.0,19.28
367,347.0,2021.0,2021-11-28,11.0,12.0,44.117,TAM,0.0,IND,25.0,...,0.0,0.0,0.0,0.0,11.04,0.2,0.0,0.0,-0.0,11.24
368,348.0,2021.0,2021-12-05,12.0,13.0,44.124,TAM,0.0,ATL,38.0,...,0.0,0.0,0.0,0.0,28.72,-0.1,0.0,0.0,-0.0,28.62
369,349.0,2021.0,2021-12-12,13.0,14.0,44.131,TAM,1.0,BUF,31.0,...,1.0,6.0,0.0,0.0,22.52,1.6,6.0,0.0,-0.0,30.12
370,350.0,2021.0,2021-12-19,14.0,15.0,44.138,TAM,1.0,NOR,26.0,...,0.0,0.0,1.0,1.0,6.56,0.2,0.0,0.0,-2.0,7.76
371,351.0,2021.0,2021-12-26,15.0,16.0,44.145,TAM,0.0,CAR,18.0,...,0.0,0.0,0.0,0.0,13.28,1.1,0.0,0.0,-0.0,14.38
372,352.0,2021.0,2022-01-02,16.0,17.0,44.152,TAM,0.0,NYJ,34.0,...,0.0,0.0,0.0,0.0,26.40,0.0,0.0,0.0,-0.0,26.40
373,353.0,2021.0,2022-01-09,17.0,18.0,44.159,TAM,1.0,CAR,29.0,...,0.0,0.0,0.0,0.0,25.04,0.0,0.0,0.0,-0.0,25.04
375,354.0,2022.0,2022-09-11,1.0,1.0,45.039,TAM,0.0,DAL,18.0,...,0.0,0.0,0.0,0.0,10.48,-0.1,0.0,0.0,-0.0,10.38
376,355.0,2022.0,2022-09-18,2.0,2.0,45.046,TAM,0.0,NOR,18.0,...,0.0,0.0,1.0,1.0,11.60,-0.2,0.0,0.0,-2.0,12.40


In [244]:
defenses = pd.read_csv('./Defenses.csv')
defenses

,Year,RANK,NAME,PTS,FPTS,SACKS,INT,TO,PTS.1,PLAYS,...,RUSH YDS,RUSH ATT,RUSH TD,TD,RZ ATT,RZ TD,RZ TD%,1D,3D%,4D%
0,2022,1,TAM,6,45,11,5,8,27,191,...,238,63,0,3,5,3,60.0,44,30.2,33.3
1,2022,2,DEN,2,28,9,1,4,36,160,...,244,56,0,3,4,1,25.0,47,26.5,25.0
2,2022,3,SFO,0,25,8,3,4,37,173,...,236,84,2,5,4,3,75.0,43,32.5,0.0
3,2022,4,BUF,8,41,11,5,7,38,158,...,173,62,3,5,6,5,83.3,46,39.4,75.0
4,2022,4,JAX,0,36,7,6,8,38,176,...,165,53,0,5,7,3,42.9,51,42.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,2017,28,NYG,14,1224,27,13,19,388,1052,...,1933,464,10,44,53,29,54.7,313,39.6,29.4
188,2017,28,WAS,12,1097,41,16,23,388,1045,...,2146,472,13,41,57,29,50.9,307,36.7,60.0
189,2017,30,IND,10,1165,26,13,20,404,1026,...,1929,487,15,44,51,27,52.9,324,44.7,50.0
190,2017,31,CLE,6,1132,34,7,13,410,1026,...,1566,467,14,48,46,31,67.4,315,40.6,50.0


In [245]:
defenses['Opp'] = defenses['NAME']

In [246]:
name_rank = defenses.filter(items=['Year', 'Opp', 'RANK'])

In [247]:
name_rank.head(20)

,Year,Opp,RANK
0,2022,TAM,1
1,2022,DEN,2
2,2022,SFO,3
3,2022,BUF,4
4,2022,JAX,4
5,2022,GNB,6
6,2022,PHI,7
7,2022,DAL,8
8,2022,MIN,9
9,2022,CIN,9


In [248]:
clean_tb['Year'] = clean_tb['Year'].astype(int)

In [249]:
clean_tb

,Rk,Year,Date,G#,Week,Age,Tm,Home,Opp,Cmp,...,Total_NP_TD,Pts,Fmb,FL,passing_ppr,rushing_ppr,td_ppr,receiving_ppr,fumble_ppr,Total
0,1.0,2000,2000-09-03,1.0,1.0,NaN,NWE,1.0,TAM,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
1,2.0,2000,2000-09-11,2.0,2.0,NaN,NWE,0.0,NYJ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
2,3.0,2000,2000-09-17,3.0,3.0,NaN,NWE,1.0,MIN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
3,4.0,2000,2000-09-24,4.0,4.0,NaN,NWE,0.0,MIA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
4,5.0,2000,2000-10-01,5.0,5.0,NaN,NWE,0.0,DEN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,351.0,2021,2021-12-26,15.0,16.0,44.145,TAM,0.0,CAR,18.0,...,0.0,0.0,0.0,0.0,13.28,1.1,0.0,0.0,-0.0,14.38
372,352.0,2021,2022-01-02,16.0,17.0,44.152,TAM,0.0,NYJ,34.0,...,0.0,0.0,0.0,0.0,26.40,0.0,0.0,0.0,-0.0,26.40
373,353.0,2021,2022-01-09,17.0,18.0,44.159,TAM,1.0,CAR,29.0,...,0.0,0.0,0.0,0.0,25.04,0.0,0.0,0.0,-0.0,25.04
375,354.0,2022,2022-09-11,1.0,1.0,45.039,TAM,0.0,DAL,18.0,...,0.0,0.0,0.0,0.0,10.48,-0.1,0.0,0.0,-0.0,10.38


In [ ]:
#Need opponent rank in tb_clean = rank of opponent on defense df for each year


In [250]:
tb2 = clean_tb.merge(name_rank, on=['Year', 'Opp'], suffixes=('_left','_right'))
               
#name_rank[['NAME']], left_on='Opp', right_on='NAME', how='left')

In [251]:
tb2

,Rk,Year,Date,G#,Week,Age,Tm,Home,Opp,Cmp,...,Pts,Fmb,FL,passing_ppr,rushing_ppr,td_ppr,receiving_ppr,fumble_ppr,Total,RANK
0,273.0,2017,2017-09-07,1.0,1.0,40.035,NWE,1.0,KAN,16.0,...,0.0,0.0,0.0,10.68,0.0,0.0,0.0,-0.0,10.68,15
1,274.0,2017,2017-09-17,2.0,2.0,40.045,NWE,0.0,NOR,30.0,...,0.0,0.0,0.0,29.88,0.9,0.0,0.0,-0.0,30.78,10
2,275.0,2017,2017-09-24,3.0,3.0,40.052,NWE,1.0,HOU,25.0,...,0.0,3.0,1.0,35.12,0.6,0.0,0.0,-2.0,36.72,32
3,276.0,2017,2017-10-01,4.0,4.0,40.059,NWE,1.0,CAR,32.0,...,0.0,0.0,0.0,20.28,0.2,0.0,0.0,-0.0,20.48,11
4,277.0,2017,2017-10-05,5.0,5.0,40.063,NWE,0.0,TAM,30.0,...,0.0,1.0,1.0,14.12,0.5,0.0,0.0,-2.0,15.62,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,351.0,2021,2021-12-26,15.0,16.0,44.145,TAM,0.0,CAR,18.0,...,0.0,0.0,0.0,13.28,1.1,0.0,0.0,-0.0,14.38,21
78,353.0,2021,2022-01-09,17.0,18.0,44.159,TAM,1.0,CAR,29.0,...,0.0,0.0,0.0,25.04,0.0,0.0,0.0,-0.0,25.04,21
79,352.0,2021,2022-01-02,16.0,17.0,44.152,TAM,0.0,NYJ,34.0,...,0.0,0.0,0.0,26.40,0.0,0.0,0.0,-0.0,26.40,32
80,354.0,2022,2022-09-11,1.0,1.0,45.039,TAM,0.0,DAL,18.0,...,0.0,0.0,0.0,10.48,-0.1,0.0,0.0,-0.0,10.38,8


In [229]:
tb2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82 entries, 0 to 81
Data columns (total 44 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rk             82 non-null     float64
 1   Year           82 non-null     int64  
 2   Date           82 non-null     object 
 3   G#             82 non-null     float64
 4   Week           82 non-null     float64
 5   Age            82 non-null     float64
 6   Tm             82 non-null     object 
 7   Home           82 non-null     float64
 8   Opp            82 non-null     object 
 9   GS             0 non-null      float64
 10  Cmp            82 non-null     float64
 11  Att            82 non-null     float64
 12  Cmp%           82 non-null     float64
 13  Yds            82 non-null     float64
 14  TD             82 non-null     float64
 15  Int            82 non-null     float64
 16  Rate           82 non-null     float64
 17  Sk             82 non-null     float64
 18  Sk_Yds      

In [265]:
tb2['Month'] = tb2.Date.str[5:7]

In [271]:
tb3 = tb2.filter(items = ['Year', 'G#', 'Week', 'Age', 'Tm', 'Home', 'Opp', 'Att', 'Cmp%', 'Sk', 'Ru_Att', 'RANK', 'Dome', 'Total'])

In [273]:
tb3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82 entries, 0 to 81
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    82 non-null     int64  
 1   G#      82 non-null     float64
 2   Week    82 non-null     float64
 3   Age     82 non-null     float64
 4   Tm      82 non-null     object 
 5   Home    82 non-null     float64
 6   Opp     82 non-null     object 
 7   Att     82 non-null     float64
 8   Cmp%    82 non-null     float64
 9   Sk      82 non-null     float64
 10  Ru_Att  82 non-null     float64
 11  RANK    82 non-null     int64  
 12  Dome    82 non-null     int64  
dtypes: float64(8), int64(3), object(2)
memory usage: 9.0+ KB
